In [155]:
import pandas as pd
import numpy as np
import requests
import ast

pd.options.display.max_columns = None

# Pair Programming ETL Transformación I

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

    1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

    2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:

    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [156]:
# Cargamos el df de los ataques
df = pd.read_csv("../data/05-tiburon_4.csv", index_col = 0)

In [157]:
df.head()

,type,country,activity,species_,mes,deceso,genero,year,age
0,Boating,usa,Paddling,White,Jun,N,F,2018.0,57.0
1,Unprovoked,brazil,Swimming,Tiger,Jun,Y,M,2018.0,18.0
2,Unprovoked,usa,Walking,Bull,May,N,M,2018.0,15.0
3,Provoked,australia,Feeding sharks,Grey,May,N,M,2018.0,32.0
4,Invalid,england,Fishing,Unspecified,May,N,M,2018.0,21.0


In [158]:
# Cargamos el dataframe del clima
df_clima = pd.read_csv("../data/00-datos_clima.csv", index_col = 0)

In [159]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,usa,39.78373,-100.445882
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,usa,39.78373,-100.445882


In [160]:
# Hacemos una lista solo con los paises que hay en el df del clima
lista_paises = df_clima["country"].unique().tolist()
lista_paises

['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

In [161]:
# Creamos un dataframe filtrando el df de los ataques con los paises que están en clima
df_pais = df[df["country"].isin(lista_paises)]
df_pais.head()

,type,country,activity,species_,mes,deceso,genero,year,age
0,Boating,usa,Paddling,White,Jun,N,F,2018.0,57.0
2,Unprovoked,usa,Walking,Bull,May,N,M,2018.0,15.0
3,Provoked,australia,Feeding sharks,Grey,May,N,M,2018.0,32.0
6,Unprovoked,australia,Surfing,Unspecified,Apr,N,M,2018.0,60.0
8,Unprovoked,south africa,Paddle-skiing,White,Apr,N,M,2018.0,33.0


In [162]:
df_clima.dtypes

timepoint              int64
cloudcover             int64
highcloud              int64
midcloud               int64
lowcloud               int64
rh_profile            object
wind_profile          object
temp2m                 int64
lifted_index           int64
rh2m                   int64
msl_pressure           int64
prec_type             object
prec_amount            int64
snow_depth             int64
wind10m.direction      int64
wind10m.speed          int64
country               object
latitud              float64
longitud             float64
dtype: object

In [163]:
# Hacemos que dentro de la columna haya listas que podamos separar por columnas
df_clima["rh_profile"] = df_clima["rh_profile"].apply(ast.literal_eval)
df_clima["wind_profile"] = df_clima["wind_profile"].apply(ast.literal_eval)

In [164]:
# Ahora podremos separar la columna en diferentes columnas, una por cada diccionario
df_rh = df_clima["rh_profile"].apply(pd.Series)
df_rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': -2}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 4}"
1,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"


In [165]:
df_clima.shape[1]

19

In [166]:
for i in range(len(df_rh.columns)):
    #Aplicamos el apply y extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0])
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_rh[i].apply(pd.Series)["rh"])
    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
    df_clima.insert(i, nombre, valores)

In [167]:
df_clima.head(1)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,3,0,-2,-1,0,2,4,6,7,7,5,1,-1,2,2,4,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,usa,39.78373,-100.445882


In [168]:
# Hacemos lo mismo en la columna Wind_profile
df_wind = df_clima["wind_profile"].apply(pd.Series)
df_wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 210, 'speed': 2}","{'layer': '900mb', 'direction': 250, 'speed': 2}","{'layer': '850mb', 'direction': 300, 'speed': 3}","{'layer': '800mb', 'direction': 305, 'speed': 4}","{'layer': '750mb', 'direction': 315, 'speed': 4}","{'layer': '700mb', 'direction': 320, 'speed': 5}","{'layer': '650mb', 'direction': 315, 'speed': 5}","{'layer': '600mb', 'direction': 310, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 305, 'speed': 6}","{'layer': '450mb', 'direction': 310, 'speed': 6}","{'layer': '400mb', 'direction': 325, 'speed': 6}","{'layer': '350mb', 'direction': 330, 'speed': 7}","{'layer': '300mb', 'direction': 335, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 8}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
1,"{'layer': '950mb', 'direction': 345, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 355, 'speed': 3}","{'layer': '800mb', 'direction': 325, 'speed': 3}","{'layer': '750mb', 'direction': 315, 'speed': 3}","{'layer': '700mb', 'direction': 310, 'speed': 4}","{'layer': '650mb', 'direction': 310, 'speed': 5}","{'layer': '600mb', 'direction': 305, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 310, 'speed': 5}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 330, 'speed': 7}","{'layer': '350mb', 'direction': 325, 'speed': 7}","{'layer': '300mb', 'direction': 325, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 7}","{'layer': '200mb', 'direction': 340, 'speed': 8}"


In [169]:
for i in range(len(df_wind.columns)):
    #Aplicamos el apply y extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string
    nombre = "wind_direction" + str(df_wind[i].apply(pd.Series)["layer"][0])
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_wind[i].apply(pd.Series)["direction"])
    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
    df_clima.insert(i, nombre, valores)

In [170]:
for i in range(len(df_wind.columns)):
    # Hacemos lo mismo para speed
    nombre2= "wind_speed" + str(df_wind[i].apply(pd.Series)["layer"][0])
    valores2 = list(df_wind[i].apply(pd.Series)["speed"])
    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
    df_clima.insert(i, nombre2, valores2)

In [171]:
df_clima.head(2)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,2,2,3,4,4,5,5,5,5,6,6,6,7,7,8,8,210,250,300,305,315,320,315,310,305,305,310,325,330,335,335,340,3,0,-2,-1,0,2,4,6,7,7,5,1,-1,2,2,4,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,usa,39.78373,-100.445882
1,2,2,3,3,3,4,5,5,5,5,6,7,7,7,7,8,345,35,355,325,315,310,310,305,305,310,325,330,325,325,335,340,2,0,0,1,3,5,7,7,5,3,2,0,4,5,7,7,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,usa,39.78373,-100.445882


In [172]:
# Borramos las columnas rh_profile y wind_profile que ya no nos sirven
df_clima.drop(["rh_profile", "wind_profile"], axis= 1, inplace= True)

In [173]:
df_clima.head(1)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,2,2,3,4,4,5,5,5,5,6,6,6,7,7,8,8,210,250,300,305,315,320,315,310,305,305,310,325,330,335,335,340,3,0,-2,-1,0,2,4,6,7,7,5,1,-1,2,2,4,3,1,-9999,-9999,-9999,12,15,4,1026,none,0,0,270,2,usa,39.78373,-100.445882


In [174]:
# Hacemos un groupby por pais y sacamos la media
df_media_c = df_clima.groupby("country").mean()

/tmp/ipykernel_2188/2333158651.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_media_c = df_clima.groupby("country").mean()


In [175]:
df_media_c

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
australia,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,4.000000,4.031250,4.250000,4.593750,5.093750,5.671875,121.406250,115.703125,129.609375,135.546875,136.250000,133.828125,136.484375,132.421875,133.906250,129.453125,121.015625,120.546875,123.203125,138.203125,151.875000,161.015625,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.375000,2.171875,2.078125,1.546875,1.640625,4.125000,6.984375,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875,-24.776109,134.755000
new zealand,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,4.484375,4.796875,5.093750,5.406250,6.406250,7.500000,157.890625,159.921875,178.359375,191.875000,194.765625,212.500000,218.984375,223.125000,216.328125,219.843750,233.515625,241.796875,254.843750,264.609375,276.484375,288.593750,13.359375,12.906250,7.921875,5.296875,3.265625,2.671875,1.875000,1.703125,3.140625,4.640625,4.906250,5.109375,5.031250,4.046875,1.421875,-1.812500,97.5,6.734375,-9999.0,-9999.0,-9999.0,15.062500,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375,-41.500083,172.834408
papua new guinea,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,4.062500,4.218750,4.265625,4.140625,4.687500,5.078125,82.812500,82.031250,82.109375,80.390625,77.343750,79.843750,145.234375,196.953125,240.781250,260.859375,256.796875,243.281250,239.140625,220.859375,190.468750,221.562500,13.578125,11.531250,9.343750,5.812500,2.531250,0.343750,1.296875,0.031250,-1.812500,-1.906250,0.296875,1.750000,2.218750,2.218750,2.125000,1.171875,97.5,3.546875,-9999.0,-9999.0,-9999.0,25.890625,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250,-5.681607,144.248908
south africa,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,3.843750,4.140625,4.578125,5.093750,5.359375,5.968750,147.656250,142.812500,159.062500,181.484375,196.250000,202.968750,203.125000,200.468750,211.406250,215.000000,216.875000,219.218750,216.953125,212.812500,209.687500,208.437500,12.328125,8.703125,6.546875,7.421875,8.812500,9.953125,9.984375,8.296875,5.625000,4.562500,4.687500,4.953125,5.656250,6.312500,7.593750,9.390625,97.5,7.031250,-9999.0,-9999.0,-9999.0,23.421875,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875,-28.816624,24.991639
usa,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,4.968750,5.046875,5.078125,5.421875,5.640625,5.484375,222.812500,230.468750,276.875000,279.921875,271.718750,267.187500,261.562500,261.015625,267.656250,263.671875,271.640625,260.234375,240.468750,235.390625,222.500000,248.828125,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.281250,4.921875,5.046875,6.078125,6.656250,5.937500,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750,39.783730,-100.445882


In [176]:
df_pais.head(2)

,type,country,activity,species_,mes,deceso,genero,year,age
0,Boating,usa,Paddling,White,Jun,N,F,2018.0,57.0
2,Unprovoked,usa,Walking,Bull,May,N,M,2018.0,15.0


In [177]:
# Unimos el dataframe con las medias del clima por pais, con el df de los ataques
df_pais_clima = pd.merge(df_pais, df_media_c, on = "country", how= "left")

In [178]:
df_pais_clima.head()

,type,country,activity,species_,mes,deceso,genero,year,age,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,Boating,usa,Paddling,White,Jun,N,F,2018.0,57.0,2.343750,2.515625,2.953125,3.296875,3.53125,3.937500,4.296875,4.484375,4.609375,4.734375,4.96875,5.046875,5.078125,5.421875,5.640625,5.484375,222.81250,230.468750,276.875000,279.921875,271.71875,267.187500,261.562500,261.015625,267.65625,263.671875,271.640625,260.234375,240.468750,235.390625,222.5000,248.828125,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.28125,4.921875,5.046875,6.078125,6.656250,5.93750,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.81250,1022.968750,0.000000,0.0,203.75000,2.093750,39.783730,-100.445882
1,Unprovoked,usa,Walking,Bull,May,N,M,2018.0,15.0,2.343750,2.515625,2.953125,3.296875,3.53125,3.937500,4.296875,4.484375,4.609375,4.734375,4.96875,5.046875,5.078125,5.421875,5.640625,5.484375,222.81250,230.468750,276.875000,279.921875,271.71875,267.187500,261.562500,261.015625,267.65625,263.671875,271.640625,260.234375,240.468750,235.390625,222.5000,248.828125,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.28125,4.921875,5.046875,6.078125,6.656250,5.93750,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.81250,1022.968750,0.000000,0.0,203.75000,2.093750,39.783730,-100.445882
2,Provoked,australia,Feeding sharks,Grey,May,N,M,2018.0,32.0,3.671875,3.640625,3.500000,3.406250,3.37500,3.390625,3.312500,3.421875,3.703125,3.843750,4.00000,4.031250,4.250000,4.593750,5.093750,5.671875,121.40625,115.703125,129.609375,135.546875,136.25000,133.828125,136.484375,132.421875,133.90625,129.453125,121.015625,120.546875,123.203125,138.203125,151.8750,161.015625,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.37500,2.171875,2.078125,1.546875,1.640625,4.12500,6.984375,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.90625,1014.906250,2.859375,0.0,122.96875,3.296875,-24.776109,134.755000
3,Unprovoked,australia,Surfing,Unspecified,Apr,N,M,2018.0,60.0,3.671875,3.640625,3.500000,3.406250,3.37500,3.390625,3.312500,3.421875,3.703125,3.843750,4.00000,4.031250,4.250000,4.593750,5.093750,5.671875,121.40625,115.703125,129.609375,135.546875,136.25000,133.828125,136.484375,132.421875,133.90625,129.453125,121.015625,120.546875,123.203125,138.203125,151.8750,161.015625,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.37500,2.171875,2.078125,1.546875,1.640625,4.12500,6.984375,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.90625,1014.906250,2.859375,0.0,122.96875,3.296875,-24.776109,134.755000
4,Unprovoked,south africa,Paddle-skiing,White,Apr,N,M,2018.0,33.0,2.359375,2.265625,2.218750,2.140625,2.15625,2.234375,2.484375,2.781250,3.062500,3.375000,3.84375,4.140625,4.578125,5.093750,5.359375,5.968750,147.65625,142.812500,159.062500,181.484375,196.25000,202.968750,203.125000,200.468750,211.40625,215.000000,216.875000,219.218750,216.953125,212.812500,209.6875,208.437500,12.328125,8.703125,6.546875,7.421875,8.812500,9.953125,9.984375,8.296875,5.625000,4.56250,4.687500,4.95312

In [179]:
df_pais_clima.shape

(1352, 72)

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [180]:
# Guardamos
df_pais_clima.to_csv("../data/00-ataques-clima.csv")